# This code is designed to transform a chosen .poni file from pyFAI into a .yml file readable by heXRD
created: 2024 09  
creator: thomas stevens  
This code is robust up to rotation angles of +- 90 degrees as it relies on reverse engineering rotation matrices using $\arcsin$ functions.  
You should only need to edit the file names/locations in the second cell to get this to function.  

In [1]:
import pyFAI
import yaml
import calibration_transform as CT

In [2]:
# Telling the code where to find the required .poni files and what to call the resultant .yml file. The path applies to both, no file type suffix is required.
path = "../"
read_files = ['V1', 'V2']
write_file = 'Configuration_test'

In [3]:
# Setting up the dictionary for later use
data = {'beam':{'energy':0,
                'source_distance':'.inf',
                'vector':{'azimuth':90.0,
                          'polar_angle':90.0}
                },
        'detectors':{},
        'id': 'instrument',
        'oscillation_stage':{
            'chi':0.0,
            'translation':[0,0,0]
        }}

In [4]:
# Looping over the files requested
for file in read_files:
    poni = pyFAI.load(path+file+'.poni')  # Loading in the poni file
    pyFAI_to_heXRD = CT.PyFAIToHeXRD(poni)  # Initialising a class from the 'calibration_transforms.py' file
    pyFAI_to_heXRD.pyFAI_to_heXRD_params()  # This transforms into the heXRD coordinate system
    detector = pyFAI_to_heXRD.write_detector()  # This writes the required information into a dictionary called detector
    data['detectors'][file] = detector  # Detector names in the yml file match the names of the poni files
    
data['beam']['energy'] = pyFAI_to_heXRD.get_energy()

In [5]:
# Saving the data in a .yml file
with open(path+write_file+'.yml','w') as write_file:
    yaml.dump(data, write_file)
    write_file.close()